# **INF8460 A20 Project: Siamese and NLP models**

<br>

Equipe 8:


*   Cedric Sadeu 
*   Mamoudou Sacko 
*   Oumayma Messoussi 

<br>

---

<br>

In [ ]:
!pip install sent2vec transformers pytorch-pretrained-bert keybert -q # pytorch-nlp pytorch_transformers

     |████████████████████████████████| 1.4MB 12.9MB/s 
     |████████████████████████████████| 133kB 57.5MB/s 
     |████████████████████████████████| 2.9MB 50.9MB/s 
     |████████████████████████████████| 890kB 63.8MB/s 
     |████████████████████████████████| 133kB 63.0MB/s 
     |████████████████████████████████| 71kB 8.5MB/s 
     |████████████████████████████████| 71kB 7.8MB/s 
     |████████████████████████████████| 6.9MB 54.2MB/s 
ERROR: botocore 1.19.26 has requirement urllib3<1.27,>=1.25.4; python_version != "3.4", but you'll have urllib3 1.24.3 which is incompatible.
ERROR: sentence-transformers 0.3.9 has requirement transformers<3.6.0,>=3.1.0, but you'll have transformers 4.0.0 which is incompatible.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks/INF8460/Project/')

### Lecture des donnees

In [ ]:
from data_handling import *

data_path = "data"
output_path = "/content/drive/My Drive/Colab Notebooks/INF8460/Project/output"

train_data = read_data(os.path.join(data_path, "/content/drive/My Drive/Colab Notebooks/INF8460/Project/data/corpus.csv"))
# train_ids = read_questions(os.path.join(data_path, "/content/drive/My Drive/Colab Notebooks/INF8460/Project/data/train_ids.csv"))
val_ids = read_questions(os.path.join(data_path, "/content/drive/My Drive/Colab Notebooks/INF8460/Project/data/val_ids.csv"))
# test_ids = read_questions(os.path.join(data_path, "/content/drive/My Drive/Colab Notebooks/INF8460/Project/data/test.csv"))


paragraphs = [" ".join(sentence.split()).lower() for sentence in train_data[1]]
# questions = [" ".join(sentence.split()).lower() for sentence in train_ids[1]]
val_questions = [" ".join(sentence.split()).lower() for sentence in val_ids[1]]
# test_questions = [" ".join(sentence.split()).lower() for sentence in test_ids[1]]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
from keybert import KeyBERT
kw_extractor = KeyBERT('distilbert-base-nli-mean-tokens')

array_text = [val_questions[0], paragraphs[17134]]
for j in range(len(array_text)):
    keywords = kw_extractor.extract_keywords(array_text[j], keyphrase_length=2, stop_words='english')
    print("Keywords of article", str(j+1), "\n", keywords)

Keywords of article 1 
 ['architecture hyderabad', 'oldest piece', 'shahi architecture', 'qutb shahi', 'piece qutb']
Keywords of article 2 
 ['old hyderabad', 'classical persian', 'hyderabad square', 'architecture 16th', 'hyderabad ruins']


### Pretraitement

In [ ]:
from data_handling import *
pre = Preprocess()

paragraphs_tokenized = pre.preprocess_pipeline(paragraphs)
# questions_tokenized = pre.preprocess_pipeline(questions)
val_questions_tokenized = pre.preprocess_pipeline(val_questions)
# test_questions_tokenized = pre.preprocess_pipeline(test_questions)

paragraphs_text = [" ".join(sentence) for sentence in paragraphs_tokenized]
# questions_text = [" ".join(sentence) for sentence in questions_tokenized]
val_questions_text = [" ".join(sentence) for sentence in val_questions_tokenized]
# test_questions_text = [" ".join(sentence) for sentence in test_questions_tokenized]

del paragraphs_tokenized
# del questions_tokenized
del val_questions_tokenized
# del test_questions_tokenized

del paragraphs
del val_questions



---

<br>

## **1. Plongements lexicaux**

### TF-IDF

In [ ]:
from create_embeddings import buildVocab, getTfIdfReprentation, get_doc_embedded, sklearn_svd
from sklearn.feature_extraction.text import TfidfVectorizer

# paragraphs_vocab = buildVocab(paragraphs_text)
# max_feat = int(0.01 * len(paragraphs_vocab))
vectorizer = TfidfVectorizer(max_features=10000) # vocabulary=paragraphs_vocab
paragraphs_tfidf = getTfIdfReprentation(paragraphs_text, vectorizer)

val_questions_tfidf = vectorizer.transform(val_questions_text).todense()

# corpus = {'id': train_data[0], 'paragraph': paragraphs_tfidf}
# save_to_csv("corpus.csv", corpus)

# train_ids = {'id': train_ids[0], 'question': paragraphs_tfidf, 'paragraph_id': train_ids[2], 'answer': train_ids[3] }
# save_to_csv("train_ids.csv", train_ids)

### GloVe

In [ ]:
# !wget http://nlp.stanford.edu/data/glove.6B.zip
!wget http://nlp.stanford.edu/data/glove.42B.300d.zip
# !unzip glove.6B.zip
!unzip glove.42B.300d.zip
# !rm glove.6B.50d.txt
# !rm glove.6B.100d.txt
# !rm glove.6B.200d.txt

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from create_embeddings import read_from_csv, get_gloves_dict, get_lines_gloves, get_plong_corpus

path = "/content/drive/My Drive/Colab Notebooks/INF8460/Project/data/corpus.csv"
datat = read_from_csv(path)

vectorizer = CountVectorizer()
X = vectorizer.fit(paragraphs_text).vocabulary_

In [ ]:
glove_dict = get_gloves_dict() # "glove.42B.300d.txt"
key_set = set(X.keys()) & set(glove_dict.keys())
glove_dict_vocab_corpus = {key: glove_dict[key] for key in key_set}

In [ ]:
plongement_docs = get_plong_corpus(paragraphs_text, glove_dict_vocab_corpus)
plongement_queries = get_plong_corpus(val_questions_text, glove_dict_vocab_corpus)



---

<br>

## **2. Ordonnancement**



> #### LSTM Siamese text similarity



In [ ]:
!wget https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/sick2014/SICK_train.txt

In [ ]:
import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks/INF8460/Project/LSTMSiameseTextSimilarity/')

In [ ]:
from model import SiameseBiLSTM
from inputHandler import word_embed_meta_data, create_test_data
from config import siamese_config
import pandas as pd

############ Data Preperation ##########

# df = pd.read_csv('lstm-siamese-text-similarity/sample_data.csv')
df = pd.read_csv('SICK_train.txt', names=['pair_ID', 'sentence_A', 'sentence_B', 'relatedness_score', 'entailment_judgment'], 
                 skiprows=1, sep='\t')
df.drop('pair_ID', axis=1, inplace=True)
df.drop(df[df.entailment_judgment == 'CONTRADICTION'].index, inplace=True)
print(df.head())

sentences1 = list(df['sentence_A'])
sentences2 = list(df['sentence_B'])
is_similar = list(df['relatedness_score'])
del df

                                          sentence_A  ... entailment_judgment
0  A group of kids is playing in a yard and an ol...  ...             NEUTRAL
1  A group of children is playing in the house an...  ...             NEUTRAL
2  The young boys are playing outdoors and the ma...  ...          ENTAILMENT
3  The kids are playing outdoors near a man with ...  ...             NEUTRAL
4  The young boys are playing outdoors and the ma...  ...             NEUTRAL

[5 rows x 4 columns]


In [ ]:
######## Word Embedding ############

tokenizer, embedding_matrix = word_embed_meta_data(sentences1 + sentences2,  siamese_config['EMBEDDING_DIM'], glove_dict)

embedding_meta_data = {
	'tokenizer': tokenizer,
	'embedding_matrix': embedding_matrix
}

## creating sentence pairs
sentences_pair = [(x1, x2) for x1, x2 in zip(sentences1, sentences2)]
del sentences1
del sentences2

######## Training ########

class Configuration(object):
    """Dump stuff here"""

CONFIG = Configuration()

CONFIG.embedding_dim = siamese_config['EMBEDDING_DIM']
CONFIG.max_sequence_length = siamese_config['MAX_SEQUENCE_LENGTH']
CONFIG.number_lstm_units = siamese_config['NUMBER_LSTM']
CONFIG.rate_drop_lstm = siamese_config['RATE_DROP_LSTM']
CONFIG.number_dense_units = siamese_config['NUMBER_DENSE_UNITS']
CONFIG.activation_function = siamese_config['ACTIVATION_FUNCTION']
CONFIG.rate_drop_dense = siamese_config['RATE_DROP_DENSE']
CONFIG.validation_split_ratio = siamese_config['VALIDATION_SPLIT']

siamese = SiameseBiLSTM(CONFIG.embedding_dim , CONFIG.max_sequence_length, CONFIG.number_lstm_units , CONFIG.number_dense_units, CONFIG.rate_drop_lstm, CONFIG.rate_drop_dense, CONFIG.activation_function, CONFIG.validation_split_ratio)

best_model_path = siamese.train_model(sentences_pair, is_similar, embedding_meta_data, model_save_directory='./')

Embedding matrix shape: (2277, 300)
vector not found for word - dog,
vector not found for word - isn't
vector not found for word - man's
vector not found for word - black,
vector not found for word - corndogs
vector not found for word - little,
vector not found for word - young,
vector not found for word - white,
vector not found for word - small,
vector not found for word - girl,
vector not found for word - red,
vector not found for word - woman,
vector not found for word - uniform,
vector not found for word - child,
vector not found for word - rope,
vector not found for word - wagon,
vector not found for word - field,
vector not found for word - boy,
vector not found for word - backbends
vector not found for word - necklace,
vector not found for word - tree,
vector not found for word - pine,
vector not found for word - snow,
vector not found for word - blue,
vector not found for word - gray,
vector not found for word - big,
vector not found for word - dog's
vector not found for word 

In [ ]:
######## Testing Q/Q pairs ########

from operator import itemgetter
from keras.models import load_model

model = load_model(best_model_path)

test_sentence_pairs = [('A man with a jersey is dunking the ball at a basketball game', 'The ball is being dunked by a man with a jersey at a basketball game'),
                       ('Two dogs are fighting', 'Two dogs are wrestling and hugging'), 
                       ('A group of kids is playing in a yard and an old man is standing in the background', 'A group of boys in a yard is playing and a man is standing in the background'), 
                       ('What can make Physics easy to learn?','How can you make physics easy to learn?'),
                       ('How many times a day do a clocks hands overlap?','What does it mean that every time I look at the clock the numbers are the same?')]

test_data_x1, test_data_x2, leaks_test = create_test_data(tokenizer,test_sentence_pairs,  siamese_config['MAX_SEQUENCE_LENGTH'])

preds = list(model.predict([test_data_x1, test_data_x2, leaks_test], verbose=1).ravel())
results = [(x, y, z) for (x, y), z in zip(test_sentence_pairs, preds)]
results.sort(key=itemgetter(2), reverse=True)
print(results)

1/1 [==============================] - 0s 7ms/step
[('A man with a jersey is dunking the ball at a basketball game', 'The ball is being dunked by a man with a jersey at a basketball game', 4.673317), ('A group of kids is playing in a yard and an old man is standing in the background', 'A group of boys in a yard is playing and a man is standing in the background', 3.9694657), ('Two dogs are fighting', 'Two dogs are wrestling and hugging', 3.3925214), ('What can make Physics easy to learn?', 'How can you make physics easy to learn?', 3.1973097), ('How many times a day do a clocks hands overlap?', 'What does it mean that every time I look at the clock the numbers are the same?', 1.8688544)]


In [ ]:
for i, doc in enumerate(paragraphs_text):
    test_sentence_pairs = [(doc, passage) for passage in set(paragraphs_text[:i]+paragraphs_text[i+1:])]

    test_data_x1, test_data_x2, leaks_test = create_test_data(tokenizer, test_sentence_pairs, siamese_config['MAX_SEQUENCE_LENGTH'])

    preds = list(model.predict([test_data_x1, test_data_x2, leaks_test], verbose=1).ravel())
    results = [(x, y, z) for (x, y), z in zip(test_sentence_pairs, preds)]
    results.sort(key=itemgetter(2), reverse=True)
    
    print(doc)
    print('\n')
    for line in results[:10]:
        print(line[1], line[2])

    break

2096/2096 [==============================] - 61s 29ms/step
the norman norman nourmands french normands latin normanni were the people who in the and century gave their name to normandy region in france they were descended from norse norman come from norseman raider and pirate from denmark iceland and norway who under their leader rollo agreed to swear fealty to king charles iii of west francia through generation of assimilation and mixing with the native frankish and roman gaulish population their descendant would gradually merge with the carolingian based culture of west francia the distinct cultural and ethnic identity of the norman emerged initially in the first half of the century and it continued to evolve over the succeeding century


both the ford foundation and the rockefeller foundation were heavily involved one key leader wa norman borlaug the father of the green revolution who received the nobel peace prize in he is credited with saving over billion people from starvation th



> #### MLP regression model



In [ ]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/INF8460/Project/MSMARCO/msmarco-docdev-100k.csv')
df.drop(['Unnamed: 0'], axis=1, inplace=True)
df = df.iloc[:25000,:] 
print(len(df))

df_score = df['score']
normalized_df = (df_score - df_score.min())/ (df_score.max()-df_score.min()) #df_score.std()
del df_score
print(normalized_df.head())

y = normalized_df#.multiply(1.0/df["rank"], axis="index")
# print(y.head())
del normalized_df

df.drop(['rank', 'score'], axis=1, inplace=True)

# Concatenate queries and docs in pairs

# X = df[['query', 'doc']].agg(' [sep] '.join, axis=1)
# del df

25000
0    0.761969
1    0.759416
2    0.755649
3    0.748219
4    0.746034
Name: score, dtype: float64


In [ ]:
from data_handling import *
pre = Preprocess()

paragraphs_tokenized = pre.preprocess_pipeline(df["doc"])
questions_tokenized = pre.preprocess_pipeline(df["query"])

paragraphs_text = [" ".join(sentence) for sentence in paragraphs_tokenized]
questions_text = [" ".join(sentence) for sentence in questions_tokenized]

del paragraphs_tokenized
del questions_tokenized
del df

In [ ]:
# tfidf embeddings

from create_embeddings import buildVocab, getTfIdfReprentation, get_doc_embedded, sklearn_svd
from sklearn.feature_extraction.text import TfidfVectorizer

# paragraphs_vocab = buildVocab(paragraphs_text) # taille du vocab: 495737
# max_feat = int(0.01 * len(paragraphs_vocab))
# vectorizer = TfidfVectorizer(max_features=max_feat) # vocabulary=paragraphs_vocab
msmarco_paragraphs_tfidf = getTfIdfReprentation(paragraphs_text, vectorizer) # meme vectorizer utilisé pour notre corpus

# new_question = vals_questions_text[i]
# new_question_tokenized = pre.preprocess_pipeline([new_question])
# new_question_text = [" ".join(sentence) for sentence in new_question_tokenized]

msmarco_questions_tfidf = vectorizer.transform(questions_text).todense()

In [ ]:
# glove embeddings

import pickle
from sklearn.feature_extraction.text import CountVectorizer
from create_embeddings import get_gloves_dict, get_lines_gloves, get_plong_corpus

vectorizer = CountVectorizer()
vocab_docs = vectorizer.fit(df["doc"]).vocabulary_

glove_dict = get_gloves_dict() # "glove.42B.300d.txt"
key_set = set(vocab_docs.keys()) & set(glove_dict.keys())
glove_dict_vocab_corpus = {key: glove_dict[key] for key in key_set}
del vocab_docs
plongement_docs = get_plong_corpus(df["doc"], glove_dict_vocab_corpus)
plongement_queries = get_plong_corpus(df["query"], glove_dict_vocab_corpus)

# with open('glove_query_emb_msmarco.txt', 'wb') as fp:
#     pickle.dump(plongement_queries, fp)
# del plongement_queries
# df.drop(['query'], axis=1, inplace=True)

In [ ]:
with open('/content/drive/MyDrive/Colab Notebooks/INF8460/Project/glove_query_emb_msmarco.txt', 'wb') as fp:
    pickle.dump(plongement_queries, fp)
del plongement_queries
df.drop(['query'], axis=1, inplace=True)

with open('/content/drive/MyDrive/Colab Notebooks/INF8460/Project/glove_doc_emb_msmarco.txt', 'wb') as fp:
    pickle.dump(plongement_docs, fp)
del plongement_docs
df.drop(['doc'], axis=1, inplace=True)

In [ ]:
import pickle

with open('/content/drive/MyDrive/Colab Notebooks/INF8460/Project/glove_query_emb_msmarco.txt', 'rb') as fp:
    plongement_queries = pickle.load(fp)

with open('/content/drive/MyDrive/Colab Notebooks/INF8460/Project/glove_doc_emb_msmarco.txt', 'rb') as fp:
    plongement_docs = pickle.load(fp)

In [ ]:
# bert embeddings

from scipy import spatial
from sent2vec.vectorizer import Vectorizer

vectorizer = Vectorizer()
vectorizer.bert(df["doc"])
docs_bert = vectorizer.vectors

vectorizer.bert(df["query"])
queries_bert = vectorizer.vectors

Token indices sequence length is longer than the specified maximum sequence length for this model (2831 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
import pickle

with open('bert_query_emb_msmarco.txt', 'wb') as fp:
    pickle.dump(queries_bert, fp)

with open('bert_doc_emb_msmarco.txt', 'wb') as fp:
    pickle.dump(docs_bert, fp)

In [ ]:
import tensorflow as tf
from keras.layers import Dense, Embedding, Input
from keras.models import Sequential, Model
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(np.concatenate((plongement_queries, plongement_docs), axis=1), y, test_size=0.10, random_state=42)
# X_train, X_test, y_train, y_test = train_test_split(np.concatenate((msmarco_questions_tfidf, msmarco_paragraphs_tfidf), axis=1), y, test_size=0.10, random_state=42)
# X_train, X_test, y_train, y_test = train_test_split(np.concatenate((queries_bert, docs_bert), axis=1), y, test_size=0.20, random_state=42)

def MLPregression(X, y, ndims):
    model = Sequential()
    model.add(Dense(100, input_dim=ndims, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1, activation='linear'))
    model.compile(optimizer="adam", loss="mean_squared_error", metrics=['mse'])

    # stop = EarlyStopping(monitor='val_accuracy', patience=2, restore_best_weights=True)
    model.fit(X, y, epochs=25, batch_size=100, validation_split=0.15) #, callbacks=[stop])
    return model
    
max_feat = 300
MLPmodel = MLPregression(X_train, np.array(y_train), max_feat*2)
preds = MLPmodel.predict(X_test)

MLPmodel.save('/content/drive/MyDrive/Colab Notebooks/INF8460/Project/mlp_regression_msmarco_glove_300*2.h5')

Epoch 1/25
192/192 [==============================] - 1s 4ms/step - loss: 0.0111 - mse: 0.0111 - val_loss: 0.0034 - val_mse: 0.0034
Epoch 2/25
192/192 [==============================] - 1s 3ms/step - loss: 0.0028 - mse: 0.0028 - val_loss: 0.0026 - val_mse: 0.0026
Epoch 3/25
192/192 [==============================] - 1s 3ms/step - loss: 0.0022 - mse: 0.0022 - val_loss: 0.0020 - val_mse: 0.0020
Epoch 4/25
192/192 [==============================] - 1s 3ms/step - loss: 0.0018 - mse: 0.0018 - val_loss: 0.0019 - val_mse: 0.0019
Epoch 5/25
192/192 [==============================] - 1s 3ms/step - loss: 0.0018 - mse: 0.0018 - val_loss: 0.0018 - val_mse: 0.0018
Epoch 6/25
192/192 [==============================] - 1s 3ms/step - loss: 0.0016 - mse: 0.0016 - val_loss: 0.0018 - val_mse: 0.0018
Epoch 7/25
192/192 [==============================] - 1s 3ms/step - loss: 0.0016 - mse: 0.0016 - val_loss: 0.0015 - val_mse: 0.0015
Epoch 8/25
192/192 [==============================] - 1s 3ms/step - loss: 0.

In [ ]:
print(np.array(y_test)[:5])
print(preds[:5])

[0.74325098 0.78950238 0.78060363 0.78547908 0.74975728]
[[0.7599584 ]
 [0.77603745]
 [0.79252803]
 [0.7936619 ]
 [0.75642383]]


In [ ]:
import keras
from operator import itemgetter

reconstructed_MLPmodel = keras.models.load_model('/content/drive/MyDrive/Colab Notebooks/INF8460/Project/mlp_regression_msmarco_glove_300*2.h5')

val_results = {}
for i, query in enumerate(plongement_queries): # val_questions_tfidf
    results = []
    query = query.tolist()
    test_sentence_pairs = np.matrix([query for n in range(len(plongement_docs))]) # paragraphs_tfidf
    # for j, doc in enumerate(paragraphs_tfidf):
    test_sentence_pairs = np.concatenate((test_sentence_pairs, plongement_docs), axis=1)
    preds = reconstructed_MLPmodel.predict(test_sentence_pairs)

    for j in train_data[0]:
        results.append((j, preds[j][0]))
    results.sort(key=itemgetter(1), reverse=True)
    val_results[i] = results[:1000]
        
    print(val_ids[2][i])
    correct = False
    if val_ids[2][i] != -1:
        print(val_results[i])
        for line in val_results[i]:
            # print(line[0], line[1])
            if line[0] == int(val_ids[2][i]):
                correct = True 
    print(correct)

17134.0
[(83271, 1.0017147), (32963, 1.0002354), (13147, 0.99690825), (35427, 0.99501556), (7252, 0.98948896), (3812, 0.9885626), (2293, 0.9844908), (31397, 0.98320943), (49012, 0.981814), (5113, 0.98158914), (13142, 0.97977966), (3601, 0.9747692), (68873, 0.97109145), (3981, 0.970495), (14436, 0.96938306), (4694, 0.9682901), (13144, 0.9674437), (13169, 0.9653858), (13174, 0.9653855), (79777, 0.96391326), (5969, 0.9635116), (6504, 0.96326554), (26284, 0.9628693), (76099, 0.96239895), (4714, 0.9610033), (4451, 0.95820254), (3465, 0.95779675), (75907, 0.9572347), (53798, 0.95601046), (46150, 0.95590055), (83137, 0.9556515), (15028, 0.9555307), (57286, 0.95513546), (7123, 0.9548602), (986, 0.95359796), (25323, 0.9533777), (2547, 0.9528138), (21863, 0.952201), (6488, 0.9521107), (4754, 0.9511115), (77550, 0.95049906), (13170, 0.9504017), (37161, 0.9502999), (13205, 0.94987464), (38328, 0.9491012), (19443, 0.94879085), (13161, 0.94874096), (12778, 0.9485541), (83165, 0.9482688), (16861, 0.9

KeyboardInterrupt: ignored